In [3]:
# Import packages
import pandas as pd
import numpy as np
import os
import numpy as np
from scipy import stats
from sklearn.preprocessing import RobustScaler

In [25]:
df = pd.read_csv('UTR_official.csv')
df.head()

,seqname,source,feature,start,end,strand,ref_gene_id,reference_id,transcript_length,molecularSubtype,Start_UTRref,End_UTRref,UTR_length,ref_UTR_length,UTR_diff,UTR_log,UTR_type,z_score_diff,scaled_diff
0,chr22,StringTie,transcript,42692115,42720870,-,A4GALT,XM_005261647.3,28755,HER2+,42692114,42692889,775,775,0,6.652863,no change,0.055836,0.0
1,chr12,StringTie,transcript,31959449,31987302,+,RESF1,XM_006719099.3,27853,HER2+,31987212,31987302,90,90,0,4.499810,no change,0.055836,0.0
2,chr4,StringTie,transcript,56907900,56935844,+,REST,NM_005612.5,27944,HER2+,56932152,56935844,3692,3692,0,8.213924,no change,0.055836,0.0
3,chr19,StringTie,transcript,7669049,7670455,+,RETN,NM_020415.4,1406,HER2+,7670349,7670455,106,106,0,4.663439,no change,0.055836,0.0
4,chr19,StringTie,transcript,18588798,18592337,+,REX1BD,NM_001100418.2,3539,HER2+,18592180,18592337,157,157,0,5.056246,no change,0.055836,0.0


In [ ]:
# Assuming you have a DataFrame named 'df' with columns 'molecularSubtype', 'gene_id', and 'utr lengths'

# Group by 'molecularSubtype' and 'gene_id', and filter out rows with the same 'utr lengths'
filtered_df = df.groupby(['molecularSubtype', 'ref_gene_id']).apply(lambda x: x.drop_duplicates(subset='UTR_length'))

# Reset the index of the filtered DataFrame
filtered_df = filtered_df.reset_index(drop=True)

# Print the resulting DataFrame
print(filtered_df)

In [30]:
# # Calculate the Z-scores
# df['z_score_diff'] = stats.zscore(df['UTR_diff'])

# # Create a RobustScaler object
# scaler = RobustScaler()

# # Extract one column as a NumPy array
# column_array = df['UTR_diff'].values

# # Reshape the array to a 2D array
# reshaped_array = column_array.reshape(-1, 1)

# # Fit the scaler to the dataset and transform the data
# df['scaled_diff'] = scaler.fit_transform(reshaped_array)

In [26]:
# Group df by molecular subtype and save to grouped df
grouped = df.groupby('molecularSubtype')


# Create separate dataframes for each group and save them to individual files
for group_name, group_data in grouped:
    # Extract the log_UTR column for the group
    log_UTR_data = group_data['UTR_log']
    
    # Create a new dataframe with the log_UTR column
    group_df = pd.DataFrame({'UTR_log': log_UTR_data})
    
    # Save the dataframe to a file named by type
    filename = f"{group_name}_UTR_log.csv"
    group_df.to_csv(filename, index=False)

In [27]:
lumA = pd.read_csv("luminal A_UTR_log.csv")
lumB = pd.read_csv("luminal B_UTR_log.csv")
her = pd.read_csv("HER2+_UTR_log.csv")
triple = pd.read_csv("triple negative_UTR_log.csv")
control = pd.read_csv('Uninvolved_UTR_log.csv')

In [7]:
def shapiro_wilk_normality_test(data, alpha=0.05):
    """
    Performs the Shapiro-Wilk normality test on the given data.

    Args:
        data: 1-D array-like or iterable. The data to be tested for normality.
        alpha: float, optional. The significance level. Default is 0.05.

    Returns:
        None. Prints the test result indicating if the data follows a normal distribution or not.
    """
    statistic, p_value = stats.shapiro(data)

    print(f"Shapiro-Wilk Test: statistic = {statistic}, p-value = {p_value}")

    if p_value > alpha:
        print("The data follows a normal distribution")
    else:
        print("The data does not follow a normal distribution")

In [28]:
shapiro_wilk_normality_test(lumA)

Shapiro-Wilk Test: statistic = 0.9873398542404175, p-value = 7.969729071004432e-30
The data does not follow a normal distribution


C:\Users\natdzi3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\stats\_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [29]:
shapiro_wilk_normality_test(lumB)

Shapiro-Wilk Test: statistic = 0.98769211769104, p-value = 2.3530074674308128e-29
The data does not follow a normal distribution


In [30]:
shapiro_wilk_normality_test(triple)

Shapiro-Wilk Test: statistic = 0.9881249070167542, p-value = 3.4104060510124286e-29
The data does not follow a normal distribution


In [31]:
shapiro_wilk_normality_test(her)

Shapiro-Wilk Test: statistic = 0.9883401393890381, p-value = 1.0132591882438323e-28
The data does not follow a normal distribution


In [38]:
shapiro_wilk_normality_test(control)

Shapiro-Wilk Test: statistic = 0.9872274994850159, p-value = 4.48163994355407e-28
The data does not follow a normal distribution


In [11]:
datasets = [lumA['UTR_log'], lumB['UTR_log'], her['UTR_log'], triple['UTR_log'], control['UTR_log']]

from scipy.stats import levene
stats.levene(lumA['UTR_log'], lumB['UTR_log'], her['UTR_log'], triple['UTR_log'], control['UTR_log'])


LeveneResult(statistic=1.1791196458589035, pvalue=0.3176581299160864)

In [32]:
stats.bartlett(lumA['UTR_log'], lumB['UTR_log'], her['UTR_log'], triple['UTR_log'], control['UTR_log'])

BartlettResult(statistic=3.710680989900556, pvalue=0.4465743405759024)

In [13]:
from scipy.stats import mannwhitneyu

def compare_datasets(control, test):
    """
    Performs the Mann-Whitney U test to compare two datasets.

    Args:
        control: array-like. The control dataset.
        test: array-like. The test dataset.

    Returns:
        None. Prints the test result indicating if the datasets are significantly different.
    """
    statistic, p_value = mannwhitneyu(control, test, alternative='two-sided')

    alpha = 0.05  # Significance level

    print(f"Mann-Whitney U Test - Statistic = {statistic}, p-value = {p_value}")
    if p_value < alpha:
        print("The datasets are significantly different")
    else:
        print("The datasets are not significantly different")

In [ ]:
compare_datasets(control['UTR_length'], triple['UTR_length'])

In [34]:
compare_datasets(control['UTR_log'], lumA['UTR_log'])

Mann-Whitney U Test - Statistic = 50106802.0, p-value = 0.673520942946905
The datasets are not significantly different


In [35]:
compare_datasets(control['UTR_log'], lumB['UTR_log'])

Mann-Whitney U Test - Statistic = 49808369.0, p-value = 0.5310614845107089
The datasets are not significantly different


In [36]:
compare_datasets(control['UTR_log'], her['UTR_log'])

Mann-Whitney U Test - Statistic = 50503644.5, p-value = 0.10222487669703482
The datasets are not significantly different


In [37]:
# Group df by molecular subtype and save to grouped df
grouped = df.groupby('molecularSubtype')


# Create separate dataframes for each group and save them to individual files
for group_name, group_data in grouped:
    # Extract the log_UTR column for the group
    log_UTR_data = group_data['scaled_diff']
    
    # Create a new dataframe with the log_UTR column
    group_df = pd.DataFrame({'scaled_diff': log_UTR_data})
    
    # Save the dataframe to a file named by type
    filename = f"{group_name}_scaled_diff.csv"
    group_df.to_csv(filename, index=False)

In [38]:
lumA = pd.read_csv("luminal A_scaled_diff.csv")
lumB = pd.read_csv("luminal B_scaled_diff.csv")
her = pd.read_csv("HER2+_scaled_diff.csv")
triple = pd.read_csv("triple negative_scaled_diff.csv")
control = pd.read_csv('Uninvolved_scaled_diff.csv')

In [39]:
compare_datasets(control['scaled_diff'], her['scaled_diff'])

Mann-Whitney U Test - Statistic = 50440379.5, p-value = 0.04130589120422983
The datasets are significantly different


In [40]:
compare_datasets(control['scaled_diff'], lumA['scaled_diff'])

Mann-Whitney U Test - Statistic = 50986618.0, p-value = 0.00039512378320918673
The datasets are significantly different


In [41]:
compare_datasets(control['scaled_diff'], triple['scaled_diff'])

Mann-Whitney U Test - Statistic = 52042938.0, p-value = 2.4461750987930217e-05
The datasets are significantly different


In [42]:
compare_datasets(control['scaled_diff'], lumB['scaled_diff'])

Mann-Whitney U Test - Statistic = 50876175.0, p-value = 9.195179657557307e-06
The datasets are significantly different


In [43]:
stats.levene(lumA['scaled_diff'], lumB['scaled_diff'], her['scaled_diff'], triple['scaled_diff'], control['scaled_diff'])

LeveneResult(statistic=0.7269009286957053, pvalue=0.5734091345668928)

In [44]:
from scipy.stats import kruskal

# Perform the Kruskal-Wallis test
statistic, p_value = kruskal(control, lumA, lumB, her, triple)

# Print the results
print("Kruskal-Wallis statistic:", statistic)
print("p-value:", p_value)

Kruskal-Wallis statistic: [26.20915891]
p-value: [2.87153158e-05]
